In [1]:
def get_matrix(area: str):
    return list(map(lambda line: list(line), area.split("\n")))

def get_region(matrix, coordinates):
    area = []
    fx, fy = coordinates
    
    plant = matrix[fy][fx]

    def survey(coordinates):
        x,y = coordinates
        region = [(x, y)]
        area.append((x, y))
    
        if x > 0:
            if matrix[y][x-1] == plant:
                next_area = (x-1, y)
                if next_area not in area:
                    # print("matrix[y][x-1]")
                    region.extend(survey(next_area))
            
        if x < len(matrix[y])-1:
            if matrix[y][x+1] == plant:
                next_area = (x+1, y)
                if next_area not in area:
                    # print("matrix[y][x+1]")
                    region.extend(survey(next_area))
    
        if y > 0:
            if matrix[y-1][x] == plant:
                next_area = (x, y-1)
                if next_area not in area:
                    # print("matrix[y-1][x]")
                    region.extend(survey(next_area))
    
        if y < len(matrix)-1:
            if matrix[y+1][x] == plant:
                next_area = (x, y+1)
                if next_area not in area:
                    # print("matrix[y+1][x]")
                    region.extend(survey(next_area))

        return region

    return survey(coordinates)


def get_perimeter(region: list[int]):
    perimeter = 0
    for x,y in region:
        count = 4
        edges = [
            (x-1, y),
            (x+1, y),
            (x, y-1),
            (x, y+1)
        ]
        
        for edge in edges:
            if edge in region:
                count -= 1

        perimeter += count
    return perimeter

def get_regions(matrix: list[list[str]]):
    seen = []
    regions = []
    
    for y in range(len(matrix)):
        for x in range(len(matrix)):
            if (x,y) in seen:
                continue
            region = get_region(matrix, (x,y))
            seen.extend(region)
            regions.append(region)

    return regions

with open("input.txt", "r") as f:
    garden = f.read().strip()

matrix = get_matrix(garden)

regions = get_regions(matrix)

out = 0
for region in regions:
    perimeter = get_perimeter(region)
    out += len(region) * get_perimeter(region)

out

1451030

Correct: `1451030`

In [6]:
from colorama import Fore, Style
from itertools import cycle

def render_matrix(matrix):
    return "\n".join(["".join(line) for line in matrix])

def get_outer_blocks(region: list[int]):
    outer = []
    perimeter = 0
    for x,y in region:
        count = 4
        edges = [
            (x-1, y),
            (x+1, y),
            (x, y-1),
            (x, y+1)
        ]
        
        for edge in edges:
            if edge in region:
                count -= 1

        if count != 0:
            outer.append((x,y))
    return outer


def get_next_step(x, y, position):
    if position == "^":
        return x, y-1
    
    elif position == ">":
        return x+1, y
        
    elif position == "<":
        return x-1, y
        
    elif position == "v":
        return x, y+1


def get_turn_position(current):
    return {
        "^": "<",
        ">": "^",
        "v": ">",
        "<": "v"
    }[current]

def get_new_position(current):
    directions = ["^", ">", "v", "<", "^"]
    return directions[directions.index(current)+1]

def find_region_boundaries(region):
    x_values, y_values = zip(*region)
    return (min(x_values), max(x_values)),(min(y_values), max(y_values)),

def find_first_point(region):
    x_values, y_values = zip(*region)
    top_y = min(y_values)
    top_x = min([point[0] if point[1] == top_y else max(x_values) for point in region])
    return top_x, top_y

def find_sides(region):
    first_point = find_first_point(region)
    x, y = first_point
    # region = get_outer_blocks([*region])
    # print(f"{region=}")
    position = "^"
    sides = 0
    steps = []
    cx,cy = x, y
    count = 0
    while True:
        nx,ny = cx,cy

        if count > 0 and (cx, cy) == first_point and position == "^":
            # print("❌ End")
            break

        count += 1

        steps.append((cx,cy))
        # print(f"{cx=} {cy=} {position=} {sides=}")
        
        # Turn on left
        tx,ty = get_next_step(cx,cy, get_turn_position(position))
        if (tx,ty) in region:
            position = get_turn_position(position)
            sides += 1
            cx,cy = tx,ty
            # print("🔥 Side detected")
            continue
        
        nx,ny = get_next_step(nx,ny, position)

        if (nx,ny) not in region:
            position = get_new_position(position)
            sides += 1
            # print("🟡 Regular turn")
            continue

        cx,cy = nx,ny

    # print(f"{sides=}")
    return sides
    

# garden = """
# ..................
# ......XXXXXXX.....
# ....XXXXXX........
# ......XXXXXXX.....
# ......XXXX........
# .........XX.......
# ..................
# """.strip()


# garden = """
# ......................
# ........OOOOOOO.......
# ........O.............
# ........OOOO..........
# ........O........XXX..
# ........OOOOOOO...XXX.
# ..........X........X..
# """.strip()

garden = """
VVVVVVVVLLLLLLLLLL
CCCCCNBBBBIIIIIIII
CCCCQBBBBBIIIIIIII
CCCCCBBBBIIIIIIIII
CCCCBBBBBIIIIIIIII
CCCCCBBBBIIIIIIIII
CCCCCBBBBBBIIIIPII
CCCCCCCBBBIIIIPPII
""".strip()

# with open("input.txt", "r") as f:
#     garden = f.read().strip()


matrix = get_matrix(garden)
regions = get_regions(matrix)

colors = cycle([Fore.GREEN, Fore.RED, Fore.YELLOW, Fore.BLUE, Fore.MAGENTA, Fore.CYAN, Fore.BLACK])

price = 0
for i, region in enumerate(regions):
    color = next(colors)
    for x,y in region: # get_outer_blocks(region):
        matrix[y][x] = color + matrix[y][x] + Fore.RESET

    sides = find_sides(region)
    print(f"{sides=} {len(region)=}")
    price += sides * len(region)

print()
print(f"{price=}")
print()
print(render_matrix(matrix))

sides=4 len(region)=8
sides=14 len(region)=35
sides=4 len(region)=1
sides=18 len(region)=31
sides=4 len(region)=1

price=1088

VVVVVVVVLLLLLLLLLL
CCCCCNBBBBIIIIIIII
CCCCQBBBBBIIIIIIII
CCCCCBBBBIIIIIIIII
CCCCBBBBBIIIIIIIII
CCCCCBBBBIIIIIIIII
CCCCCBBBBBBIIIIPII
CCCCCCCBBBIIIIPPII


# Incorrect

- `836380` That's not the right answer; your answer is too low.